In [ ]:
! cp -r /content/drive/MyDrive/sft_santacoder1b /content/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install datasets peft transformers trl accelerate bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 0. imports
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from transformers import AutoTokenizer, HfArgumentParser, TrainingArguments

from trl import DPOTrainer

def dpo_data(train_or_val):

    dataset = load_dataset(
        "Dahoas/full-hh-rlhf",
        split = "train",
        use_auth_token=True
    )

    original_columns = dataset.column_names

    def return_prompt_and_responses(samples):
        return {
            "prompt": [prompt for prompt in samples["prompt"]],
            "chosen": samples["chosen"],
            "rejected": samples["rejected"],
        }

    return dataset.map(
        return_prompt_and_responses,
        batched=True,
        remove_columns=original_columns,
    )


if __name__ == "__main__":

    # 1. load a pretrained model
    model = AutoPeftModelForCausalLM.from_pretrained(
        "/content/sft_santacoder1b",
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        load_in_4bit=True,
    )
    model.config.use_cache = False

    model_ref = AutoPeftModelForCausalLM.from_pretrained(
        "/content/sft_santacoder1b/",
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        load_in_4bit=True,
    )
    tokenizer = AutoTokenizer.from_pretrained("/content/sft_santacoder1b/")
    tokenizer.pad_token = tokenizer.eos_token

    train_dataset = dpo_data("train")
    train_dataset = train_dataset.filter(
        lambda x: len(x["prompt"]) + len(x["chosen"]) <= 256
        and len(x["prompt"]) + len(x["rejected"]) <= 256
    )
    eval_dataset = dpo_data("val")
    eval_dataset = eval_dataset.filter(
        lambda x: len(x["prompt"]) + len(x["chosen"]) <= 256
        and len(x["prompt"]) + len(x["rejected"]) <= 256
    )

    training_args = TrainingArguments(
        per_device_train_batch_size=2,
        max_steps=505,
        logging_steps=10,
        save_steps=500,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        learning_rate=2e-4,
        evaluation_strategy="steps",
        eval_steps=100,
        output_dir="dpo_santacoder1b",
        report_to="tensorboard",
        lr_scheduler_type="cosine",
        warmup_steps=2,
        optim="paged_adamw_32bit",
        fp16=True,
        remove_unused_columns=False,
        run_name="dpo_llama2",
    )

    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        target_modules=["c_attn", "c_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

    dpo_trainer = DPOTrainer(
        model,
        model_ref,
        args=training_args,
        beta=0.1,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        peft_config=peft_config,
        max_prompt_length=128,
        max_length=256,
    )

    # 6. train
    dpo_trainer.train()
    dpo_trainer.save_model("dpo_santacoder1b")

    # 7. save
    output_dir = os.path.join("dpo_santacoder1b", "final_checkpoint")
    dpo_trainer.model.save_pretrained(output_dir)

Some weights of GPTBigCodeForCausalLM were not initialized from the model checkpoint at TabbyML/SantaCoder-1B and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPTBigCodeForCausalLM were not initialized from the model checkpoint at TabbyML/SantaCoder-1B and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/datasets/load.py:2072: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/112052 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12451 [00:00<?, ? examples/s]

Map:   0%|          | 0/112052 [00:00<?, ? examples/s]

Filter:   0%|          | 0/112052 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2072: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=True' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
100,0.597100,0.650092,-1.181460,-1.624547,0.633621,0.443087,-71.345589,-75.123474,-1.451947,-1.539659
200,0.667100,0.600564,-1.053160,-1.510184,0.661292,0.457023,-70.201950,-73.840469,-1.306081,-1.388318
300,0.641800,0.582740,-1.118413,-1.755315,0.688112,0.636902,-72.653259,-74.492996,-1.494221,-1.572122
400,0.657600,0.563669,-1.009875,-1.650142,0.696679,0.640268,-71.601540,-73.407608,-1.410650,-1.487489
500,0.515300,0.561103,-0.840255,-1.480818,0.700192,0.640563,-69.908287,-71.711403,-1.403130,-1.479807


OSError: ignored

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
model = AutoModelForCausalLM.from_pretrained(
        "/content/sft_santacoder1b/final_merged_checkpoint/", return_dict=True, torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(model, "/content/dpo_santacoder1b/final_checkpoint/")
model.eval()
model = model.merge_and_unload()

model.save_pretrained("/content/dpo_santacoder1b/final_merged_checkpoint")
tokenizer.save_pretrained("/content/dpo_santacoder1b/final_merged_checkpoint")
model.push_to_hub("dpo-santacoder1b")
tokenizer.push_to_hub("dpo-santacoder1b")

In [ ]:
! cp -r /content/dpo_santacoder1b /content/drive/MyDrive/SantaCoder-DPO